In [1]:
# 导入数据
using JLD2, DataFrames, DataFramesMeta, FileIO, CSV, Models, StatsBase, HypothesisTests

@load "/Users/dddd1007/project2git/cognitive_control_model/data/output/RLModels/model_selection/2021-01-07-145437final_selection_correct_CCC.jld2"

include("/Users/dddd1007/project2git/cognitive_control_model/scripts/import_all_data.jl")

┌ Info: Precompiling JLD2 [033835bb-8acc-5ee8-8aae-3f567f8a3819]
└ @ Base loading.jl:1278
┌ Info: Precompiling Models [73df4adf-0718-4152-b75d-f87bfd26a255]
└ @ Base loading.jl:1278
┌ Info: Precompiling HypothesisTests [09f84164-cd44-5f33-b23f-e6b0d136a0d5]
└ @ Base loading.jl:1278


Dict{String,String} with 4 entries:
  "response"    => "Response"
  "RT"          => "RT"
  "corrections" => "Type"
  "sub_tag"     => "Subject"

In [2]:
# 定义工具函数

## 提取特定被试的估计后参数
function extract_CCC_optim_params(eval_results, subname)
    return eval_results[subname][5][:optim_param]
end

## 添加被试单次trial的冲突程度信息
function add_conflict_list(sub_dataframe, env, subinfo, opt_params)
    model_result = model_recovery(env, subinfo, opt_params, model_type = :_1a1d1CCC)
    conflict_list = model_result[:conflict_list]
    weight_history = model_result[:options_weight_history]
    sub_dataframe.if_over_CCC = conflict_list .> opt_params[:CCC]
    weight_history_table = DataFrame(weight_history, [:ll,:lr,:rl,:rr])
    sub_dataframe = hcat(sub_dataframe, weight_history_table)
    return sub_dataframe
end

add_conflict_list (generic function with 1 method)

In [25]:
# 给所有被试添加 conflict 程度信息
temp_dataframe_stake = []

for subname in collect(keys(eval_results))
    each_sub_data = @where(all_data, :Subject .== subname)
    env, subinfo = Models.RLModels.init_env_sub(each_sub_data, env_idx_dict, sub_idx_dict)
    opt_params = extract_CCC_optim_params(eval_results, subname)
    each_sub_data = add_conflict_list(each_sub_data, env, subinfo, opt_params) 
    push!(temp_dataframe_stake, each_sub_data)
end

dataframe_with_CCC_tag = temp_dataframe_stake[1]

for i in 2:length(temp_dataframe_stake)
    dataframe_with_CCC_tag = vcat(dataframe_with_CCC_tag, temp_dataframe_stake[i])
end

# 导出数据到R中分析
CSV.write("/Users/dddd1007/project2git/cognitive_control_model/data/output/summary/subdata_with_CCC.csv", dataframe_with_CCC_tag)

The env and sub info of sub25_LuSihan is generated!
The env and sub info of sub31_TanPeixian is generated!
The env and sub info of sub14_MaKexin is generated!
The env and sub info of sub04_YiXiangjie is generated!
The env and sub info of sub24_HuangQuan is generated!
The env and sub info of sub33_LiangWenyan is generated!
The env and sub info of sub10_YangFeixian is generated!
The env and sub info of sub09_LiZiping is generated!
The env and sub info of sub20_GuoTao is generated!
The env and sub info of sub05_WuXiaohang is generated!
The env and sub info of sub11_ZengWenpei is generated!
The env and sub info of sub12_ChenRuiyi is generated!
The env and sub info of sub28_HuangDajian is generated!
The env and sub info of sub30_YanXingguang is generated!
The env and sub info of sub07_LiangNvjun is generated!
The env and sub info of sub08_Luoluo is generated!
The env and sub info of sub18_WeiSiwen is generated!
The env and sub info of sub21_LiBiying is generated!
The env and sub info of sub

"/Users/dddd1007/project2git/cognitive_control_model/data/output/summary/subdata_with_CCC.csv"

## 初步查看被试在反应时上的差异

当前的假设是, 被试在冲突较强时, 应该会采用不同的学习率. 这意味着当前的冲突较强时, 被试采取了较高的 learning rate 来更新 S-R 的价值权重, 那么之后相同的行为, 应该会存在更快的反应时. 因此我们计算 前-后 为相同的 S-R 的 trial 间 RT 的差, 查看该差值在 conflict 小于或大于 CCC threshold 后是否存在不同.

LoadError: MethodError: no method matching ExactPermutationTest(::Array{Float64,1}, ::Array{Float64,1})
Closest candidates are:
  ExactPermutationTest(::AbstractArray{R,1}, ::AbstractArray{S,1}, !Matched::Function) where {R<:Real, S<:Real} at /Users/dddd1007/.julia/packages/HypothesisTests/40Llk/src/permutation.jl:21